# Tutorial descargando datos de la web

En este tutorial veremos como recuperar datos desde la web. Revisaremos algunos módulos de Python para descargar y "entender" la web, y la funcionalidad que tiene Pandas para leer dataframes desde distintos tipos de datos.

## Los datos

Utilizaremos el sitio web de la BBC sobre los resultados de las [Olimpiadas Río 2016](http://www.bbc.com/sport/olympics/rio-2016/). El sitio incluye diaria sobre los diferentes eventos deportivos, así como tambien resultados y los países que compiten.

El objetivo es recuperar los eventos deportivos que podrían ser de interés para el público chileno. 

## HTML
HTML es el lenguaje que se utiliza para crear páginas web. El contenido y la apariciencia de un sitio web se especifica mediante etiquetas anidadas. Por ejemplo, la etiqueta ``<title>...</title>`` se utiliza para denotar el título de una página web (es el texto que vemos en un TAB del navegador).

![html](images/html.png)

El contenido de la página está dentro de las etiquetas ``<body> .... </body>``. Dentro de esa etiqueta se pueden crear tablas con datos, o párrafos con texto.

Más información sobre HTML en https://en.wikipedia.org/wiki/HTML

## Cómo comenzar

Lo primero es recolectar información sobre como funciona el sitio web objetivo. A simple vista, el sitio de la BBC contiene una sección con toda la programación de las olimpiadas, dedicando una URL para cada día: http://www.bbc.com/sport/olympics/rio-2016/schedule

Para obtener la información de un día particular se puede especifica el año, mes y día con el formato YYYY-MM-DD. Por ejemplo, http://www.bbc.com/sport/olympics/rio-2016/schedule/2016-08-07 recupera los eventos del día 7 de agosto de 2016.

![bbcrender](images/bbc_render.png)

Para ver las etiquetas HTML del sitio puedes utilizar la funcion **Inspect Element**. Por ejemplo, acá vemos el código html para los eventos de Arquería. 

![bbc_html](images/bbc_html.png)

El sitio posee muuuucho contenido, es por eso que muchas etiquetas anidadas!

## Descargando los datos

Ya tenemos un poco más claro donde están los datos que buscamos. 


In [1]:
import pandas as pd

Ahora asignaremos la url asociada a cada día de las olimpiadas, siguiendo el formato que encontramos anteriormente.

In [2]:
url = 'http://www.bbc.com/sport/olympics/rio-2016/schedule/2016-08-07'

La función .strftime permite crear string siguiendo el formato que definamos nosotros. Más info en https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.dt.strftime.html

Para descargar las páginas usaremos el módulo [**requests**](http://docs.python-requests.org/en/master/) (aunque existen varios otros!). Recuerda instalarlon con 
```bash
pip install requests
```

In [3]:
import requests
webpage = requests.get(url)
webpage.text[:1000]

'    <!DOCTYPE html>  <html id="sport-html" class="b-reith-sans-font no-js no-enhanced no-touch no-font-face no-av no-app no-csscolumns no-css-transitions no-css-2d-transforms no-flexbox no-svg"  lang="en"> <head> <title>Schedule - Rio 2016 - Olympic Games - BBC Sport</title> <meta http-equiv="X-UA-Compatible" content="IE=edge"/> <meta name="description" content="Schedule for the Olympic Games 2016" /> <meta name="keywords" content="Rio 2016, Olympics 2016, Olympic Games, schedule, BBC, BBC Sport" />  <meta property="og:site_name" content="BBC Sport"/>     <meta charset="UTF-8"/>  <link rel="canonical" href="http://www.bbc.com/sport/olympics/rio-2016/schedule/2016-08-07" />   <link rel="alternate" href="android-app://bbc.mobile.sport.ww/http/www.bbc.com/sport/olympics/rio-2016/schedule/2016-08-07" />    <link rel="alternate" hreflang="en" href="http://www.bbc.com/sport/olympics/rio-2016/schedule/2016-08-07" /> <link rel="alternate" hreflang="en-gb" href="http://www.bbc.co.uk/sport/olym

La variable webpage.text contiene el html de la url que descargamos. Ahora veremos como acceder al contenido que buscamos!

## Magical Pandas


Pandas trae un módulo para extraer contenido desde html. Es bastante rudimentario, pero es la primera opción que debes probar.

El método ``read_html(...)`` recibe como entrada un string html, o la dirección web de una página, y devuelve una lista de DataFrames con las tablas que están en el HTML.

In [4]:
tablas = pd.read_html(webpage.text) 
len(tablas)

21

Pandas descubrió 21 tablas. Cada una corresponde a un deporte. La primera tabla corresponde a los juegos de arquería:

![bbc_tabla](images/bbc_tables.png)

In [5]:
tablas[0]

,0,1,2
0,13:00 - 13:25,Women's Team 1/8 Eliminations MEX V GEO,Results
1,13:25 - 13:50,Women's Team 1/8 Eliminations JPN V UKR,Results
2,13:50 - 14:15,Women's Team 1/8 Eliminations BRZ V ITA,Results
3,14:15 - 14:40,Women's Team 1/8 Eliminations IND V COL,Results
4,18:00 - 18:25,Women's Team Quarterfinal MEX V TPE,Results
5,18:25 - 18:50,Women's Team Quarterfinal KOR V JPN,Results
6,18:50 - 19:15,Women's Team Quarterfinal CHN V ITA,Results
7,19:15 - 19:40,Women's Team Quarterfinal IND V RUS,Results
8,19:43 - 20:08,Women's Team Semifinal KOR V TPE,Results
9,20:11 - 20:36,Women's Team Semifinal ITA V RUS,Results


In [5]:
tablas[1]

,0,1,2
0,02:30 - 04:16,Men's Preliminary Round Group A VEN V SER,Results
1,16:00 - 17:31,Women's Preliminary Round Group B US V SEN,Results
2,18:15 - 20:16,Men's Preliminary Round Group B BRZ V LTU,Results
3,18:15 - 19:59,Women's Preliminary Round Group B SER V SPA,Results
4,21:30 - 23:04,Women's Preliminary Round Group A AUS V TUR,Results
5,23:00 - 00:52,Men's Preliminary Round Group B CRO V SPA,Results
6,23:45 - 01:33,Women's Preliminary Round Group A FRA V BLR,Results


Funciona! Pero parcialmente... también queremos automatizar la detección del tipo de evento asociado a cada tabla. Para eso ocuparemos el módulo [BeautifoulSoup](https://www.crummy.com/software/BeautifulSoup/).

## Recuperando información desde el html

El módulo [BeautifoulSoup](https://www.crummy.com/software/BeautifulSoup/) permite interpretar el html y extraer la información que contienen los tags anidados. Nota: La tecnología para recuperar información usando la estructura anidada se llama [parsing](https://en.wikipedia.org/wiki/Parsing), y es lo que el computador utiliza para interpretar las instrucciones que escribimos en Python o HTML.

El módulo recibe como entrada un string con el html y el parser que se desea utilizar. Sugiero utilizar el parser html5. Recuerden instalarlos con:

```bash
pip install beautifulsoup4
pip install html5
```

In [6]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(webpage.text, 'html5lib')

Para identificar la etiqueta asociada al tipo de evento examinaremos el html usando Firefox y la herramienta **Inspect Element**:

![bbcli](images/bbc_litag.png)

El deporte de cada evento está en el tag ``<span class="gel-pica-bold"> ... </li>``, con la función ``find_all`` podemos recuperar todas las apariciones del tag:

In [7]:
sports = soup.find_all("span", class_="gel-pica-bold") # retorna una lista
len(sports)

21

También son 21 :). Para extraera la información de cada etiqueta usamos ``.text``:

In [8]:
sports[0].text

' Archery '

Ahora solo debemos asignar el deporte a cada DataFrame que recuperamos anteriormente con Pandas.

## Enriqueciendo los datos

Ahora que ya tenemos la información de eventos por deporte, podemos combinar la información en los dataframes que obtuvimos con Pandas.

In [9]:
tablas[0].assign(sport = sports[0].text)

,0,1,2,sport
0,13:00 - 13:25,Women's Team 1/8 Eliminations MEX V GEO,Results,Archery
1,13:25 - 13:50,Women's Team 1/8 Eliminations JPN V UKR,Results,Archery
2,13:50 - 14:15,Women's Team 1/8 Eliminations BRZ V ITA,Results,Archery
3,14:15 - 14:40,Women's Team 1/8 Eliminations IND V COL,Results,Archery
4,18:00 - 18:25,Women's Team Quarterfinal MEX V TPE,Results,Archery
5,18:25 - 18:50,Women's Team Quarterfinal KOR V JPN,Results,Archery
6,18:50 - 19:15,Women's Team Quarterfinal CHN V ITA,Results,Archery
7,19:15 - 19:40,Women's Team Quarterfinal IND V RUS,Results,Archery
8,19:43 - 20:08,Women's Team Semifinal KOR V TPE,Results,Archery
9,20:11 - 20:36,Women's Team Semifinal ITA V RUS,Results,Archery


In [10]:
df = pd.concat([t.assign(sport = s.text.strip()) 
                for t, s in zip(tablas, sports)])
df.head()

,0,1,2,sport
0,13:00 - 13:25,Women's Team 1/8 Eliminations MEX V GEO,Results,Archery
1,13:25 - 13:50,Women's Team 1/8 Eliminations JPN V UKR,Results,Archery
2,13:50 - 14:15,Women's Team 1/8 Eliminations BRZ V ITA,Results,Archery
3,14:15 - 14:40,Women's Team 1/8 Eliminations IND V COL,Results,Archery
4,18:00 - 18:25,Women's Team Quarterfinal MEX V TPE,Results,Archery


In [11]:
df.shape

(299, 4)

Ahora terminos de limpiar el dataset. Nos falta:
* Eliminar la columna resultados
* Cambiar el nombre de las columnas
* Recuperar la hora de inicio y término de cada evento.
* Asignar el día en que ocurre el evento

In [12]:
df.columns

Index([0, 1, 2, 'sport'], dtype='object')

In [13]:
df = df.drop(2, axis='columns')

In [14]:
df = df.rename(columns={0:'time', 1:'desc'})
df.head()

,time,desc,sport
0,13:00 - 13:25,Women's Team 1/8 Eliminations MEX V GEO,Archery
1,13:25 - 13:50,Women's Team 1/8 Eliminations JPN V UKR,Archery
2,13:50 - 14:15,Women's Team 1/8 Eliminations BRZ V ITA,Archery
3,14:15 - 14:40,Women's Team 1/8 Eliminations IND V COL,Archery
4,18:00 - 18:25,Women's Team Quarterfinal MEX V TPE,Archery


Usando los métodos para procesar strings, podemos recuperar la hora de inicio y término de cada evento:

In [15]:
df.head().time.str.split('-', expand=True)[0].map(lambda x: pd.to_datetime('2016-08-27 {}:00'.format(x.strip())))

0   2016-08-27 13:00:00
1   2016-08-27 13:25:00
2   2016-08-27 13:50:00
3   2016-08-27 14:15:00
4   2016-08-27 18:00:00
Name: 0, dtype: datetime64[ns]

Sin embargo, las horas de la página web de la BBC están en el horario de Londres, así que lo ajustaremos:

In [15]:
df.head().time.str.split('-', expand=True)[0].map(lambda x: pd.to_datetime('2016-08-27 {}:00 +0100'.format(x.strip()),utc=True))

0   2016-08-27 12:00:00+00:00
1   2016-08-27 12:25:00+00:00
2   2016-08-27 12:50:00+00:00
3   2016-08-27 13:15:00+00:00
4   2016-08-27 17:00:00+00:00
Name: 0, dtype: datetime64[ns, UTC]

Y también podemos asignar el día asociado a cada evento:

In [16]:
df['event_start'] = df.time.str.split('-', expand=True)[0].map(lambda x: pd.to_datetime('2016-08-27 {}:00 +0100'.format(x.strip()),utc=True))
df['event_end'] = df.time.str.split('-', expand=True)[1].map(lambda x: pd.to_datetime('2016-08-27 {}:00 +0100'.format(x.strip()),utc=True))
df = df.drop('time', axis='columns')
df.head()

,desc,sport,event_start,event_end
0,Women's Team 1/8 Eliminations MEX V GEO,Archery,2016-08-27 12:00:00+00:00,2016-08-27 12:25:00+00:00
1,Women's Team 1/8 Eliminations JPN V UKR,Archery,2016-08-27 12:25:00+00:00,2016-08-27 12:50:00+00:00
2,Women's Team 1/8 Eliminations BRZ V ITA,Archery,2016-08-27 12:50:00+00:00,2016-08-27 13:15:00+00:00
3,Women's Team 1/8 Eliminations IND V COL,Archery,2016-08-27 13:15:00+00:00,2016-08-27 13:40:00+00:00
4,Women's Team Quarterfinal MEX V TPE,Archery,2016-08-27 17:00:00+00:00,2016-08-27 17:25:00+00:00


Hay algo extraño, los eventos comienzan a medio día... Transformemoslos en el horario de Chile:

In [17]:
df['event_start'] = df.event_start.dt.tz_convert('America/Santiago')
df['event_end'] = df.event_end.dt.tz_convert('America/Santiago')
df.head()

,desc,sport,event_start,event_end
0,Women's Team 1/8 Eliminations MEX V GEO,Archery,2016-08-27 09:00:00-03:00,2016-08-27 09:25:00-03:00
1,Women's Team 1/8 Eliminations JPN V UKR,Archery,2016-08-27 09:25:00-03:00,2016-08-27 09:50:00-03:00
2,Women's Team 1/8 Eliminations BRZ V ITA,Archery,2016-08-27 09:50:00-03:00,2016-08-27 10:15:00-03:00
3,Women's Team 1/8 Eliminations IND V COL,Archery,2016-08-27 10:15:00-03:00,2016-08-27 10:40:00-03:00
4,Women's Team Quarterfinal MEX V TPE,Archery,2016-08-27 14:00:00-03:00,2016-08-27 14:25:00-03:00


## Wrapping up

Para hacer este notebook reproducible debemos descargar la pagina web para cada día del evento, guardarlos en un archivo (como datos raw/originales) y hacer una función para procesar los datos.

Primero partiremos por hacer una función que lea el html y automatice el proceso de parsing y recuperación de los datos.

In [19]:
def get_sports(html):
    # parsear html y recuperar el deporte de cada tabla
    soup = BeautifulSoup(html, 'html5lib')
    sports = soup.find_all("span", class_="gel-pica-bold") # retorna una lista
    sports = [x.text.strip() for x in sports]
    return sports

def parse_day(html, day):
    # recuperar tablas de eventos deportivos usando pandas
    tablas = pd.read_html(html) 
    sports = get_sports(html)

    # el numero de tablas debería ser el mismo que el numero de eventos
    assert( len(tablas) == len(sports) )
        
    df = pd.concat([t.assign(sport = s) for t, s in zip(tablas, sports)])

    # renombrar columnas
    df = df.drop(2, axis='columns')
    df = df.rename(columns={0:'time', 1:'desc'})

    df = df[df['time'].str.contains('-')] #solo tabblas que contengan '-'
    
    parse_date = lambda x: pd.to_datetime('{} {}:00 +0100'.format(day.strip(), x.strip()),utc=True)
    
    df['event_start'] = df.time.str.split('-', expand=True)[0].map(parse_date)
    df['event_end'] = df.time.str.split('-', expand=True)[1].map(parse_date)
    df = df.drop('time', axis='columns')
    
    df['event_start'] = df.event_start.dt.tz_convert('America/Santiago')
    df['event_end'] = df.event_end.dt.tz_convert('America/Santiago')
    
    return df

parse_day(webpage.text, '2016-08-07').head()

,desc,sport,event_start,event_end
0,Women's Team 1/8 Eliminations MEX V GEO,Archery,2016-08-07 08:00:00-04:00,2016-08-07 08:25:00-04:00
1,Women's Team 1/8 Eliminations JPN V UKR,Archery,2016-08-07 08:25:00-04:00,2016-08-07 08:50:00-04:00
2,Women's Team 1/8 Eliminations BRZ V ITA,Archery,2016-08-07 08:50:00-04:00,2016-08-07 09:15:00-04:00
3,Women's Team 1/8 Eliminations IND V COL,Archery,2016-08-07 09:15:00-04:00,2016-08-07 09:40:00-04:00
4,Women's Team Quarterfinal MEX V TPE,Archery,2016-08-07 13:00:00-04:00,2016-08-07 13:25:00-04:00


Funciona! 🎉🎉🎉

Lo que haremos será descargar el HTML para cada día de las olimpiadas, desde el 2016-08-03 hasta el 2016-08-21. Este será el dataset **raw**.

In [20]:
df = pd.DataFrame(pd.date_range('2016-08-03','2016-08-21').strftime('%Y-%m-%d'), columns=['day'])
df['url'] = ('http://www.bbc.com/sport/olympics/rio-2016/schedule/' + df.day)
df['filename'] = 'data/bbcdata/' + df.day + '.html'
df.head()

,day,url,filename
0,2016-08-03,http://www.bbc.com/sport/olympics/rio-2016/sch...,data/bbcdata/2016-08-03.html
1,2016-08-04,http://www.bbc.com/sport/olympics/rio-2016/sch...,data/bbcdata/2016-08-04.html
2,2016-08-05,http://www.bbc.com/sport/olympics/rio-2016/sch...,data/bbcdata/2016-08-05.html
3,2016-08-06,http://www.bbc.com/sport/olympics/rio-2016/sch...,data/bbcdata/2016-08-06.html
4,2016-08-07,http://www.bbc.com/sport/olympics/rio-2016/sch...,data/bbcdata/2016-08-07.html


In [21]:
def download(url, filename):
    '''
    Download a webpage, and save its content into a file.
    '''
    webpage = requests.get(url)
    f = open(filename, 'wt')
    f.write(webpage.text)
    f.close()

Ahora verificamos si el archivo con la página se guardó correctamente:

In [22]:
download(df.url[0], df.filename[0])
!head -n1 data/bbcdata/2016-08-03.html

    <!DOCTYPE html>  <html id="sport-html" class="b-reith-sans-font no-js no-enhanced no-touch no-font-face no-av no-app no-csscolumns no-css-transitions no-css-2d-transforms no-flexbox no-svg"  lang="en"> <head> <title>Schedule - Rio 2016 - Olympic Games - BBC Sport</title> <meta http-equiv="X-UA-Compatible" content="IE=edge"/> <meta name="description" content="Schedule for the Olympic Games 2016" /> <meta name="keywords" content="Rio 2016, Olympics 2016, Olympic Games, schedule, BBC, BBC Sport" />  <meta property="og:site_name" content="BBC Sport"/>     <meta charset="UTF-8"/>  <link rel="canonical" href="http://www.bbc.com/sport/olympics/rio-2016/schedule/2016-08-03" />   <link rel="alternate" href="android-app://bbc.mobile.sport.ww/http/www.bbc.com/sport/olympics/rio-2016/schedule/2016-08-03" />    <link rel="alternate" hreflang="en" href="http://www.bbc.com/sport/olympics/rio-2016/schedule/2016-08-03" /> <link rel="alternate" hreflang="en-gb" href="http://www.bbc.co.uk/sport/olymp

Ahora que funciona, descargaremos la pagina de cada día:

In [23]:
# descargar todos los sitios

for row in df.itertuples():
    print('Downloading', row.url)
    download(row.url, row.filename)

Y ahora procesaremos cada uno de los archivos descargados con la función de parsing:

In [25]:
def process_day(filename, day):
    try:
        return parse_day(open(filename).read(), day)
    except:
        raise Exception('Error en día {}'.format(day))

In [26]:
test_df = [process_day(row.filename, row.day) for row in df.itertuples()]
test_df = pd.concat(test_df)
test_df.head()

Exception: Error en día 2016-08-06

Mmmm, uno de los días genera un error, veamos que puede ser.

In [27]:
file_error = open('data/bbcdata/2016-08-06.html').read()
len(pd.read_html(file_error)), len(get_sports(file_error))

(20, 21)

Ahá! El número de deportes para el 6 de agosto es mayor al número de tablas con datos! Veamos que sucede:

In [28]:
get_sports(file_error)

['Opening Ceremony',
 'Archery',
 'Basketball',
 'Boxing',
 'Cycling',
 'Equestrian',
 'Fencing',
 'Football',
 'Gymnastics',
 'Handball',
 'Hockey',
 'Judo',
 'Rowing',
 'Rugby Sevens',
 'Shooting',
 'Swimming',
 'Table Tennis',
 'Tennis',
 'Volleyball',
 'Water Polo',
 'Weightlifting']

Ahá!! Parece que el problema es una tabla que no existe... Modifiquemos la funcion ``get_sports`` para que no considere eventos con nombre "Opening Ceremony":

In [29]:
def get_sports(html):
    # parsear html y recuperar el deporte de cada tabla
    soup = BeautifulSoup(html, 'html5lib')
    sports = soup.find_all("span", class_="gel-pica-bold") # retorna una lista
    sports = [x.text.strip() for x in sports if ('Opening Ceremony' not in x.text)]
    return sports

In [30]:
get_sports(file_error)

['Archery',
 'Basketball',
 'Boxing',
 'Cycling',
 'Equestrian',
 'Fencing',
 'Football',
 'Gymnastics',
 'Handball',
 'Hockey',
 'Judo',
 'Rowing',
 'Rugby Sevens',
 'Shooting',
 'Swimming',
 'Table Tennis',
 'Tennis',
 'Volleyball',
 'Water Polo',
 'Weightlifting']

Ahora sí! Veamos que tal nos va con la nueva función para procesar los datos:

In [31]:
%%time
all_events_df = pd.concat([process_day(row.filename, row.day) for row in df.itertuples()])
all_events_df.head()

CPU times: user 41 s, sys: 473 ms, total: 41.4 s
Wall time: 41.9 s


In [33]:
all_events_df.shape

(3414, 4)

In [34]:
all_events_df.head()

,desc,sport,event_start,event_end
0,Women's First Round - Group E SWE V SA,Football,2016-08-03 12:00:00-04:00,2016-08-03 13:53:00-04:00
1,Women's First Round - Group F CAN V AUS,Football,2016-08-03 14:00:00-04:00,2016-08-03 15:52:00-04:00
2,Women's First Round - Group E BRZ V CHN,Football,2016-08-03 15:00:00-04:00,2016-08-03 16:51:00-04:00
3,Women's First Round - Group F ZIM V GER,Football,2016-08-03 17:00:00-04:00,2016-08-03 18:53:00-04:00
4,Women's First Round - Group G US V NZ,Football,2016-08-03 18:00:00-04:00,2016-08-02 19:50:00-04:00


Todo bien!!

In [35]:
all_events_df.sport.value_counts()

Wrestling                410
Boxing                   273
Fencing                  269
Tennis                   217
Badminton                207
Judo                     204
Volleyball               182
Table Tennis             172
Archery                  156
Taekwondo                152
Sailing                  120
Athletics                118
Gymnastics                92
Swimming                  86
Hockey                    76
Handball                  76
Basketball                76
Rowing                    72
Rugby Sevens              68
Water Polo                66
Football                  58
Cycling                   58
Canoeing                  52
Shooting                  51
Equestrian                33
Weightlifting             29
Diving                    16
Modern Pentathlon         10
Golf                       8
Synchronised Swimming      5
Triathlon                  2
Name: sport, dtype: int64

# Extra: procesamiento en paralelo :)

Extraer la información de las 19 páginas de las olimpiadas toma unos 40 segundos, más o menos 2 segundos por página. No es mucho tiempo, pero si quisieramos procesar todas los artículos de Wikipedia (son unos 5 millones) el proceso tomaría 117 días.

Una forma de acelerar el proceso (sin adentrarse en las oscuras aguas de la optimización de las librerías que usamos) es procesar los datos en paralelo. A continuación usaremos el modulo [Joblib](https://pythonhosted.org/joblib/parallel.html) que permite realizar tareas en paralelo de una forma muy sencilla.

In [37]:
from joblib import Parallel, delayed

Joblib ejecuta funciones en paralelo. El nombre de la función a ejecutar se debe indicar como parámetro de la función ``delayed(...)``. La cantidad de procesadores a usar se indica con la variable ``n_jobs``, en este caso usaremos dos. La función ``process_day(...)`` que definimos anteriormente es la que se ejectutará en paralelo:

In [39]:
pardf = Parallel(n_jobs=2, verbose=1)(delayed(process_day)(row.filename, row.day) for row in df.itertuples())
pardf = pd.concat(pardf)
pardf.head()

[Parallel(n_jobs=2)]: Done  19 out of  19 | elapsed:   23.1s finished


,desc,sport,event_start,event_end
0,Women's First Round - Group E SWE V SA,Football,2016-08-03 12:00:00-04:00,2016-08-03 13:53:00-04:00
1,Women's First Round - Group F CAN V AUS,Football,2016-08-03 14:00:00-04:00,2016-08-03 15:52:00-04:00
2,Women's First Round - Group E BRZ V CHN,Football,2016-08-03 15:00:00-04:00,2016-08-03 16:51:00-04:00
3,Women's First Round - Group F ZIM V GER,Football,2016-08-03 17:00:00-04:00,2016-08-03 18:53:00-04:00
4,Women's First Round - Group G US V NZ,Football,2016-08-03 18:00:00-04:00,2016-08-02 19:50:00-04:00


Usando dos procesadores logramos reducir el tiempo de ejecución a la mitad, casi 1 segundo por cada archivo html.

# Resumen

En este tutorial revisamos cómo extraer información desde la web usando Python. Lo primero es definir que tipo de información necesitamos e identificar desde donde podemos recuperarla.
Lo siguiente es descargar y procesar los datos. Sugiero seguir estos pasos para hacer el proceso reproducible:

1. Revisar si Pandas puede extraer datos desde los documentos html.
2. Si no se puede, utilizar BeautifoulSoup y diseñar a mano las reglas para extraer información que necesitas.
3. Descargar y guardar los archivos html. Así no dependemos de acceso a la red si debemos re-diseñar las reglas.
4. Crear un Notebook con todo el proceso. Un buen código es el que tu equipo entiende.
5. (opcional) Si el tiempo de ejecución es excesivo, hazlo en paralelo.

Si necesitas extraer información de un sitio web completo te sugiero que revises Scrapy https://doc.scrapy.org/en/latest/intro/tutorial.html

# 🎉🎉🎉 